In [1]:
import torch
import numpy as np

from datasets import VOCDectectionDataset
from sklearn.metrics import average_precision_score
from models import *
from utils import *
from config import *
from tqdm.notebook import tqdm

In [2]:
# propose_way = "edge_box"
year='2007'
pretrained='vgg16'
model = Combined_VGG16()
model.to(cfg.DEVICE)

checkpoints = torch.load(cfg.PATH.PT_PATH + "1S_norm_WholeModel_2007_vgg16_50.pt")
model.load_state_dict(checkpoints['whole_model_state_dict'])


traindata = VOCDectectionDataset("~/data/", year, 'trainval', use_corloc=True, small_box=False)
traindata_loader = data.DataLoader(traindata, 1, shuffle=False, num_workers=4, pin_memory=True)


In [3]:
positives = np.array([0 for _ in range(20)]) # 
hits = np.array([0 for _ in range(20)]) # 
n = 0

with torch.no_grad():
    model.eval();
    for n_imgs, gt, n_regions, region in tqdm(traindata_loader, "Evaluation"):
        n += 1
        region = region.to(cfg.DEVICE)
        avg_scores = torch.zeros((len(region[0]), 20), dtype=torch.float32)
        scales = len(cfg.DATA.SCALES)
        k = cfg.K
        for i in range(2 * scales):
            per_img = n_imgs[i].to(cfg.DEVICE)
            per_region = n_regions[i].to(cfg.DEVICE)
            ref_scores1, ref_scores2, ref_scores3, proposal_scores = model(per_img, per_region)
            avg_scores += (ref_scores1 + ref_scores2 + ref_scores3)[:, :-1].detach().cpu() / k
        avg_scores /= 2 * scales
        
        gt = gt[0]
        gt_box = gt[:, :4]
        gt_box = gt_box.to(cfg.DEVICE)
    
    
        target = [0 for _ in range(len(VOC_CLASSES))]
        gt_target = gt[:, -1]
        for t in gt_target:
            target[int(t)] = 1.0
        gt_target = torch.Tensor(target).to(cfg.DEVICE)
        
        region = region[0].cpu()
        
        for c in range(20):
            if gt_target[c] == 0.0:
                continue
            else:
                positives[c] += 1
                cls_region = region
                cls_scores = avg_scores[:, c]
                
#                 nms_filter = nms(cls_region, cls_scores, 0.3)
#                 cls_region = cls_region[nms_filter]
#                 cls_scores = cls_scores[nms_filter]
                
                most_confident_one = torch.argmax(cls_scores)
                most_confident_region = region[most_confident_one:most_confident_one+1].to(cfg.DEVICE)
#                 most_confident_region = region[nms_filter][most_confident_one:most_confident_one+1]
                IOU = one2allbox_iou(most_confident_region, gt_box)
                if (IOU >= 0.5).sum() >= 1.0:
                    hits[c] += 1
#         if n == 3:
#             break
        if n % 500 == 0:
            print(f"CorLoc is {hits/positives}")
    corloc = hits/positives
    print(f"CorLoc is {corloc}")
    print(f"mean CorLoc is {(corloc).sum()/20}")

CorLoc is [0.88461538 0.76923077 0.43333333 0.47826087 0.28571429 0.94117647
 0.86585366 0.51724138 0.37254902 0.72222222 0.56       0.675
 0.51851852 0.95454545 0.19680851 0.52631579 0.72727273 0.5
 0.71428571 0.39130435]
CorLoc is [0.82       0.71698113 0.54411765 0.42105263 0.40425532 0.75
 0.89655172 0.53846154 0.44210526 0.74193548 0.41304348 0.67058824
 0.5        0.94       0.205      0.44736842 0.61904762 0.53191489
 0.65909091 0.2962963 ]
CorLoc is [0.79104478 0.71052632 0.55339806 0.42857143 0.40277778 0.74468085
 0.89473684 0.53061224 0.4887218  0.78       0.44615385 0.6484375
 0.47560976 0.94666667 0.19763514 0.41269841 0.68571429 0.57575758
 0.63888889 0.32876712]
CorLoc is [0.80232558 0.7        0.56849315 0.4109589  0.39795918 0.72727273
 0.875      0.53076923 0.45454545 0.77192982 0.45121951 0.6091954
 0.53982301 0.92783505 0.17336683 0.39175258 0.70454545 0.57291667
 0.64705882 0.3960396 ]
CorLoc is [0.80530973 0.73170732 0.57303371 0.47777778 0.42063492 0.71428571
 0.

CorLoc is [0.81932773 0.73251029 0.59090909 0.40883978 0.38934426 0.77956989
 0.86956522 0.57566766 0.43820225 0.73758865 0.455      0.59144893
 0.60278746 0.90204082 0.17131474 0.39591837 0.75       0.54585153
 0.63984674 0.4296875 ]
 
mean CorLoc is 0.5912710450321519